# Prediction Tasks

---

This notebook is part of the [CaTabRa GitHub repository](https://github.com/risc-mi/catabra).

This short example demonstrates the four prediction tasks supported by CaTabRa:

* [binary classification](#Binary-Classification),
* [multiclass classification](#Multiclass-Classification),
* [multilabel classification](#Multilabel-Classification), and
* [regression](#Regression).

Familiarity with CaTabRa's main data analysis workflow is assumed. A step-by-step introduction can be found in [CaTabRa Workflow](https://catabra.readthedocs.io/en/latest/jupyter/workflow.html).

## Prerequisites

In [14]:
import numpy as np
from catabra.analysis import analyze

## Binary Classification

Analyze data with a binary target, i.e., each sample belongs to one of two classes.

In [2]:
# load dataset
from sklearn.datasets import load_breast_cancer
X_binary, y_binary = load_breast_cancer(as_frame=True, return_X_y=True)

In [3]:
# add target labels to DataFrame
X_binary['diagnosis'] = y_binary

In [4]:
# split into train- and test set by adding column with corresponding values
# the name of the column is arbitrary; CaTabRa tries to "guess" which samples belong to which set based on the column name and -values
X_binary['train'] = X_binary.index <= 0.8 * len(X_binary)

In [5]:
analyze(
    X_binary,                 # table to analyze; can also be the path to a CSV/Excel/HDF5 file
    classify='diagnosis',     # name of column containing classification target
    split='train',            # name of column containing information about the train-test split (optional)
    time=1,                   # time budget for hyperparameter tuning, in minutes (optional)
    out='binary_classification'
)

[CaTabRa] ### Analysis started at 2023-02-07 10:44:35.243270
[CaTabRa] Saving descriptive statistics completed
[CaTabRa] Using AutoML-backend auto-sklearn for binary_classification
[CaTabRa] Successfully loaded the following auto-sklearn add-on module(s): xgb


/home/amaletzk/miniconda3/envs/catabra/lib/python3.9/site-packages/autosklearn/metalearning/metalearning/meta_base.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.metafeatures = self.metafeatures.append(metafeatures)
/home/amaletzk/miniconda3/envs/catabra/lib/python3.9/site-packages/autosklearn/metalearning/metalearning/meta_base.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.algorithm_runs[metric].append(runs)


[CaTabRa] New ensemble fitted:
    ensemble_val_roc_auc: 0.980337
    n_constituent_models: 1
    total_elapsed_time: 00:04
[CaTabRa] New model #1 trained:
    val_roc_auc: 0.980337
    val_accuracy: 0.927152
    val_balanced_accuracy: 0.928416
    train_roc_auc: 1.000000
    type: random_forest
    total_elapsed_time: 00:04
[CaTabRa] New ensemble fitted:
    ensemble_val_roc_auc: 0.994744
    n_constituent_models: 1
    total_elapsed_time: 00:05
[CaTabRa] New model #2 trained:
    val_roc_auc: 0.994744
    val_accuracy: 0.947020
    val_balanced_accuracy: 0.947717
    train_roc_auc: 0.996970
    type: passive_aggressive
    total_elapsed_time: 00:05
[CaTabRa] New ensemble fitted:
    ensemble_val_roc_auc: 0.994744
    n_constituent_models: 1
    total_elapsed_time: 00:06
[CaTabRa] New model #3 trained:
    val_roc_auc: 0.970098
    val_accuracy: 0.920530
    val_balanced_accuracy: 0.915458
    train_roc_auc: 1.000000
    type: gradient_boosting
    total_elapsed_time: 00:06
[CaTabRa] 

[CaTabRa] Final training statistics:
    n_models_trained: 27
    ensemble_val_roc_auc: 0.9980065241029358
[CaTabRa] Creating shap explainer
[CaTabRa] Initialized out-of-distribution detector of type Autoencoder
[CaTabRa] Fitting out-of-distribution detector...
Iteration 1, loss = 0.06377027
Iteration 2, loss = 0.03359140
Iteration 3, loss = 0.02194009
Iteration 4, loss = 0.01638331
Iteration 5, loss = 0.01398731
Iteration 6, loss = 0.01247938
Iteration 7, loss = 0.01109453
Iteration 8, loss = 0.01022823
Iteration 9, loss = 0.00959181
Iteration 10, loss = 0.00878776
Iteration 11, loss = 0.00890786
Iteration 12, loss = 0.00813948
Iteration 13, loss = 0.00792689
Iteration 14, loss = 0.00700066
Iteration 15, loss = 0.00632195
Iteration 16, loss = 0.00601902
Iteration 17, loss = 0.00576786
Iteration 18, loss = 0.00579422
Iteration 19, loss = 0.00653422
Iteration 20, loss = 0.00673960
Iteration 21, loss = 0.00594721
Iteration 22, loss = 0.00580508
Iteration 23, loss = 0.00573597
Iteration 2

## Multiclass Classification

Analyze data with a multiclass target, i.e., each sample belongs to one of `n > 2` classes.

In [19]:
# load dataset
from sklearn.datasets import load_iris
X_multiclass, y_multiclass = load_iris(as_frame=True, return_X_y=True)

In [20]:
# add target labels to DataFrame
X_multiclass['species'] = y_multiclass

In [21]:
# split into train- and test set by adding column with corresponding values
# the name of the column is arbitrary; CaTabRa tries to "guess" which samples belong to which set based on the column name and -values
X_multiclass['test'] = np.random.choice([False, True], size=len(X_multiclass), p=[0.8, 0.2])

In [22]:
X_multiclass.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,test
0,5.1,3.5,1.4,0.2,0,True
1,4.9,3.0,1.4,0.2,0,False
2,4.7,3.2,1.3,0.2,0,False
3,4.6,3.1,1.5,0.2,0,False
4,5.0,3.6,1.4,0.2,0,True


In [23]:
X_multiclass['species'].value_counts()

0    50
1    50
2    50
Name: species, dtype: int64

In [24]:
X_multiclass.groupby('test')['species'].value_counts().unstack()

species,0,1,2
test,,,
False,36,39,40
True,14,11,10


Function `analyze()` is called just as before. CaTabRa automatically treats the given data as multiclass, because the target column specified by `classify` contains more than two unique values.

In [26]:
analyze(
    X_multiclass,             # table to analyze; can also be the path to a CSV/Excel/HDF5 file
    classify='species',       # name of column containing classification target
    split='test',             # name of column containing information about the train-test split (optional)
    time=1,                   # time budget for hyperparameter tuning, in minutes (optional)
    out='multiclass_classification'
)

Output folder "/mnt/c/Users/amaletzk/Documents/CaTabRa/catabra/examples/multiclass_classification" already exists. Delete? [y/n] y
[CaTabRa] ### Analysis started at 2023-02-07 10:52:44.468626
[CaTabRa] Saving descriptive statistics completed
[CaTabRa] Using AutoML-backend auto-sklearn for multiclass_classification
[CaTabRa] Successfully loaded the following auto-sklearn add-on module(s): xgb


The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


[CaTabRa] New ensemble fitted:
    ensemble_val_accuracy: 0.973684
    n_constituent_models: 1
    total_elapsed_time: 00:05
[CaTabRa] New model #1 trained:
    val_accuracy: 0.973684
    val_balanced_accuracy: 0.974359
    train_accuracy: 1.000000
    type: random_forest
    total_elapsed_time: 00:05
[CaTabRa] New ensemble fitted:
    ensemble_val_accuracy: 0.973684
    n_constituent_models: 2
    total_elapsed_time: 00:07
[CaTabRa] New model #2 trained:
    val_accuracy: 0.973684
    val_balanced_accuracy: 0.974359
    train_accuracy: 0.987013
    type: random_forest
    total_elapsed_time: 00:07
[CaTabRa] New ensemble fitted:
    ensemble_val_accuracy: 0.973684
    n_constituent_models: 2
    total_elapsed_time: 00:07
[CaTabRa] New model #3 trained:
    val_accuracy: 0.868421
    val_balanced_accuracy: 0.871795
    train_accuracy: 0.883117
    type: passive_aggressive
    total_elapsed_time: 00:07
[CaTabRa] New ensemble fitted:
    ensemble_val_accuracy: 0.973684
    n_constituent_m

[CaTabRa] Final training statistics:
    n_models_trained: 31
    ensemble_val_accuracy: 1.0
[CaTabRa] Creating shap explainer
[CaTabRa] Initialized out-of-distribution detector of type Autoencoder
[CaTabRa] Fitting out-of-distribution detector...
Iteration 1, loss = 1.61104467
Iteration 2, loss = 1.43654067
Iteration 3, loss = 1.27854978
Iteration 4, loss = 1.13614818
Iteration 5, loss = 1.00836412
Iteration 6, loss = 0.89419353
Iteration 7, loss = 0.79261599
Iteration 8, loss = 0.70261024
Iteration 9, loss = 0.62316871
Iteration 10, loss = 0.55331055
Iteration 11, loss = 0.49209276
Iteration 12, loss = 0.43861932
Iteration 13, loss = 0.39204793
Iteration 14, loss = 0.35159479
Iteration 15, loss = 0.31653725
Iteration 16, loss = 0.28621472
Iteration 17, loss = 0.26002813
Iteration 18, loss = 0.23743812
Iteration 19, loss = 0.21796231
Iteration 20, loss = 0.20117202
Iteration 21, loss = 0.18668841
Iteration 22, loss = 0.17417852
Iteration 23, loss = 0.16349008
Iteration 24, loss = 0.15

Note how the performance metrics reported during training differ from those in binary classification. This is also reflected in the detailed performance reports in subdirectory `eval/`.

## Multilabel Classification

Analyze data with a multilabel target, i.e., each sample belongs to an arbitrary subset of `n >= 2` classes.

In [28]:
# load dataset
from sklearn.datasets import fetch_openml
X_multilabel, y_multilabel = fetch_openml(data_id=40595, as_frame=True, return_X_y=True)

In [37]:
# add target labels to DataFrame
X_multilabel = X_multilabel.join(y_multilabel == 'TRUE')

In [38]:
# split into train- and test set by adding column with corresponding values
# the name of the column is arbitrary; CaTabRa tries to "guess" which samples belong to which set based on the column name and -values
X_multilabel['train'] = X_multilabel.index <= 0.8 * len(X_multilabel)

In [39]:
X_multilabel.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,Att11,Att12,Att13,Att14,Att15,Att16,Att17,Att18,Att19,Att20,Att21,Att22,Att23,Att24,Att25,Att26,Att27,Att28,Att29,Att30,Att31,Att32,Att33,Att34,Att35,Att36,Att37,Att38,Att39,Att40,...,Att262,Att263,Att264,Att265,Att266,Att267,Att268,Att269,Att270,Att271,Att272,Att273,Att274,Att275,Att276,Att277,Att278,Att279,Att280,Att281,Att282,Att283,Att284,Att285,Att286,Att287,Att288,Att289,Att290,Att291,Att292,Att293,Att294,Beach,Sunset,FallFoliage,Field,Mountain,Urban,train
0,0.646467,0.666435,0.685047,0.699053,0.652746,0.407864,0.150309,0.535193,0.555689,0.580782,0.577094,0.390455,0.242458,0.170217,0.421797,0.428206,0.428277,0.490017,0.459252,0.350897,0.255987,0.310109,0.375018,0.437369,0.451752,0.508325,0.467347,0.567068,0.546262,0.566969,0.612951,0.621101,0.653561,0.694546,0.574777,0.710196,0.614510,0.590450,0.508313,0.645884,...,0.136652,0.136285,0.127585,0.249868,0.545665,0.252143,0.261571,0.203095,0.172747,0.239030,0.309251,0.090241,0.048767,0.085062,0.072274,0.167601,0.094636,0.258751,0.092845,0.477150,0.224848,0.102568,0.329816,0.061538,0.049615,0.068962,0.653879,0.354982,0.124074,0.157332,0.247298,0.014025,0.029709,True,False,False,False,True,False,True
1,0.770156,0.767255,0.761053,0.745630,0.742231,0.688086,0.708416,0.757351,0.760633,0.740314,0.513377,0.600421,0.542340,0.439594,0.604272,0.624697,0.642823,0.424883,0.448578,0.318076,0.209851,0.570696,0.599071,0.556610,0.556215,0.653352,0.559962,0.473784,0.636677,0.653249,0.621813,0.613890,0.596795,0.596297,0.692224,0.634007,0.605896,0.594992,0.650470,0.582844,...,0.097577,0.167246,0.193839,0.283507,0.190554,0.072342,0.111906,0.175488,0.178064,0.249890,0.085085,0.073259,0.133331,0.090761,0.138334,0.102932,0.406639,0.126982,0.046562,0.354085,0.199359,0.157326,0.051859,0.114123,0.160008,0.414088,0.361843,0.303399,0.176387,0.251454,0.137833,0.082672,0.036320,True,False,False,False,False,True,True
2,0.793984,0.772096,0.761820,0.762213,0.740569,0.734361,0.722677,0.849128,0.839607,0.812746,0.785767,0.760288,0.751835,0.754508,0.853808,0.857499,0.858505,0.864827,0.865957,0.867185,0.872483,0.955915,0.966291,0.968941,0.879657,0.716114,0.479571,0.402155,0.754620,0.775176,0.723823,0.676656,0.633313,0.552341,0.417900,0.622198,0.652387,0.648123,0.680452,0.662322,...,0.060296,0.058945,0.052964,0.062245,0.075563,0.006149,0.004046,0.006033,0.181837,0.213608,0.122532,0.035184,0.025505,0.027821,0.353377,0.073733,0.048943,0.080248,0.074113,0.051372,0.024035,0.015971,0.028559,0.047596,0.038082,0.079977,0.004901,0.003460,0.006049,0.017166,0.051125,0.112506,0.083924,True,False,False,False,False,False,True
3,0.938563,0.949260,0.955621,0.966743,0.968649,0.869619,0.696925,0.953460,0.959631,0.966320,0.972766,0.916497,0.622508,0.530428,0.963539,0.972303,0.972980,0.945388,0.609497,0.514073,0.360757,0.804240,0.827367,0.813407,0.796413,0.753638,0.696435,0.520342,0.782931,0.774347,0.750613,0.706845,0.612971,0.647101,0.645833,0.736683,0.719352,0.643989,0.705878,0.773725,...,0.008393,0.093743,0.105665,0.060825,0.025972,0.045153,0.039900,0.030980,0.448542,0.024508,0.024751,0.045848,0.020989,0.015197,0.209978,0.138788,0.031173,0.032565,0.034237,0.018757,0.082271,0.201563,0.043669,0.027527,0.016922,0.024174,0.036799,0.007694,0.009735,0.019267,0.031290,0.049780,0.090959,True,False,False,False,False,False,True
4,0.512130,0.524684,0.520020,0.504467,0.471209,0.417654,0.364292,0.562266,0.588592,0.584449,0.570074,0.551043,0.503925,0.447526,0.500117,0.539517,0.588721,0.600226,0.588937,0.562027,0.510786,0.465298,0.626580,0.649661,0.629969,0.574756,0.519651,0.445292,0.450048,0.742275,0.784539,0.903786,0.834243,0.766266,0.657113,0.276264,0.394086,0.610411,0.698119,0.743710,...,0.595821,0.207690,0.028206,0.010644,0.010589,0.138157,0.094097,0.044848,0.036629,0.046537,0.090652,0.086531,0.293732,0.221770,0.094467,0.143500,0.186763,0.074600,0.043375,0.208570,0.188324,0.413413,0.387559,0.158730,0.023177,0.129994,0.167709,0.226580,0.218534,0.198151,0.238796,0.164270,0.184290,True,Fal

Function `analyze()` is called just as before, with more than one column passed to `classify`. This tells CaTabRa to treat the data as multilabel.

In [40]:
analyze(
    X_multilabel,             # table to analyze; can also be the path to a CSV/Excel/HDF5 file
    classify=['Beach', 'Sunset', 'FallFoliage', 'Field', 'Mountain', 'Urban'],
    split='train',            # name of column containing information about the train-test split (optional)
    time=3,                   # time budget for hyperparameter tuning, in minutes (optional)
    out='multilabel_classification'
)

[CaTabRa] ### Analysis started at 2023-02-07 11:16:46.333073
[CaTabRa] Saving descriptive statistics completed
[CaTabRa] Using AutoML-backend auto-sklearn for multilabel_classification
[CaTabRa] Successfully loaded the following auto-sklearn add-on module(s): xgb
[WARNING] [2023-02-07 11:17:00,572:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 28 not found
[WARNING] [2023-02-07 11:17:00,572:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 265 not found
[WARNING] [2023-02-07 11:17:00,573:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 7 not found
[WARNING] [2023-02-07 11:17:00,573:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 220 not found
[WARNING] [2023-02-07 11:17:00,573:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 253 not found
[WARNING] [2023-02-07 11:17:00,573:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 355 not found
[

[WARNING] [2023-02-07 11:17:00,584:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 434 not found
[WARNING] [2023-02-07 11:17:00,584:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 199 not found
[WARNING] [2023-02-07 11:17:00,584:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 445 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 318 not found


The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 363 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 173 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 647 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 412 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 131 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 494 not found
[WARNING] [2023-02-07 11:17:00,585:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 500 not found
[WARNING] [2023-02-07 11:17:00,586:Client-AutoMLSMBO(1)::8d1e93df-a6d0-11ed-8083-00155da455e5] Configuration 37 not found
[WARNING] [2023-0

Iteration 41, loss = 0.01887094
Iteration 42, loss = 0.01887194
Iteration 43, loss = 0.01888870
Iteration 44, loss = 0.01891678
Iteration 45, loss = 0.01890816
Iteration 46, loss = 0.01889278
Iteration 47, loss = 0.01886188
Iteration 48, loss = 0.01885999
Iteration 49, loss = 0.01885912
Iteration 50, loss = 0.01886987
Iteration 51, loss = 0.01887337
Iteration 52, loss = 0.01885332
Iteration 53, loss = 0.01883346
Iteration 54, loss = 0.01887909
Iteration 55, loss = 0.01889231
Iteration 56, loss = 0.01883884
Training loss did not improve more than tol=0.000100 for 50 consecutive epochs. Stopping.
[CaTabRa] Out-of-distribution detector fitted.
[CaTabRa] ### Analysis finished at 2023-02-07 11:20:40.127400
[CaTabRa] ### Elapsed time: 0 days 00:03:53.794327
[CaTabRa] ### Output saved in /mnt/c/Users/amaletzk/Documents/CaTabRa/catabra/examples/multilabel_classification
[CaTabRa] ### Evaluation started at 2023-02-07 11:20:40.130001
[CaTabRa] Predicting out-of-distribution samples.
[CaTabRa] Sa

invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in true_divide
No positive samples in y_true, true positive value should be meaningless and recall is set to 1 for all thresholds
invalid value encountered in true_divide
No positive samples in y_true, true positive value should be meaningless and recall is set to 1 for all thresholds
invalid value encountered in true_divide
No positive samples in y_true, true positive value should be meaningless and recall is set to 1 for all thresholds


[CaTabRa] ### Evaluation finished at 2023-02-07 11:21:24.398684
[CaTabRa] ### Elapsed time: 0 days 00:00:44.268683
[CaTabRa] ### Output saved in /mnt/c/Users/amaletzk/Documents/CaTabRa/catabra/examples/multilabel_classification/eval


Note again how the performance metrics reported during training differ from those in binary- and multiclass classification. This is also reflected in the detailed performance reports in subdirectory `eval/`.

## Regression

Analyze data with one or more regression targets.

In [41]:
# load dataset
from sklearn.datasets import load_diabetes
X_regression, y_regression = load_diabetes(as_frame=True, return_X_y=True)

In [43]:
# add target labels to DataFrame
X_regression['disease_progression'] = y_regression

In [44]:
# split into train- and test set by adding column with corresponding values
# the name of the column is arbitrary; CaTabRa tries to "guess" which samples belong to which set based on the column name and -values
X_regression['train'] = X_regression.index <= 0.8 * len(X_regression)

In [45]:
X_regression.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,disease_progression,train
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0,True
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0,True
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0,True
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0,True
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0,True


Function `analyze()` is again called as before, the only difference being that keyword argument `classify` is replaced by `regress`.

In [46]:
analyze(
    X_regression,             # table to analyze; can also be the path to a CSV/Excel/HDF5 file
    regress=['disease_progression'],   # name(s) of target column(s)
    split='train',            # name of column containing information about the train-test split (optional)
    time=1,                   # time budget for hyperparameter tuning, in minutes (optional)
    out='regression'
)

[CaTabRa] ### Analysis started at 2023-02-07 11:26:30.909830
[CaTabRa] Saving descriptive statistics completed
[CaTabRa] Using AutoML-backend auto-sklearn for regression
[CaTabRa] Successfully loaded the following auto-sklearn add-on module(s): xgb


The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


[CaTabRa] New ensemble fitted:
    ensemble_val_r2: 0.346560
    n_constituent_models: 1
    total_elapsed_time: 00:14
[CaTabRa] New model #1 trained:
    val_r2: 0.346560
    val_mean_absolute_error: 47.753839
    val_mean_squared_error: 3699.549517
    train_r2: 0.924566
    type: random_forest
    total_elapsed_time: 00:14
[CaTabRa] New model #2 trained:
    val_r2: -0.000265
    val_mean_absolute_error: 62.965668
    val_mean_squared_error: 5663.151483
    train_r2: 0.998678
    type: gaussian_process
    total_elapsed_time: 00:15
[CaTabRa] New ensemble fitted:
    ensemble_val_r2: 0.347943
    n_constituent_models: 2
    total_elapsed_time: 00:19
[CaTabRa] New model #3 trained:
    val_r2: 0.065712
    val_mean_absolute_error: 60.908616
    val_mean_squared_error: 5289.613941
    train_r2: 0.997329
    type: gaussian_process
    total_elapsed_time: 00:19
[CaTabRa] New ensemble fitted:
    ensemble_val_r2: 0.388678
    n_constituent_models: 2
    total_elapsed_time: 00:21
[CaTabRa]

[CaTabRa] Fitting out-of-distribution detector...
Iteration 1, loss = 0.25534312
Iteration 2, loss = 0.14401352
Iteration 3, loss = 0.09340346
Iteration 4, loss = 0.06996843
Iteration 5, loss = 0.05760558
Iteration 6, loss = 0.05068051
Iteration 7, loss = 0.04561450
Iteration 8, loss = 0.04126809
Iteration 9, loss = 0.03752003
Iteration 10, loss = 0.03445358
Iteration 11, loss = 0.03203054
Iteration 12, loss = 0.03026330
Iteration 13, loss = 0.02909024
Iteration 14, loss = 0.02833517
Iteration 15, loss = 0.02799340
Iteration 16, loss = 0.02790263
Iteration 17, loss = 0.02800053
Iteration 18, loss = 0.02816715
Iteration 19, loss = 0.02833172
Iteration 20, loss = 0.02843938
Iteration 21, loss = 0.02845107
Iteration 22, loss = 0.02837497
Iteration 23, loss = 0.02823050
Iteration 24, loss = 0.02807415
Iteration 25, loss = 0.02792351
Iteration 26, loss = 0.02779502
Iteration 27, loss = 0.02770411
Iteration 28, loss = 0.02762860
Iteration 29, loss = 0.02758793
Iteration 30, loss = 0.02757642

The performance metrics reported during training differ from those in classification. This is also reflected in the detailed performance reports in subdirectory `eval/`.